<a href="https://colab.research.google.com/github/gemma24/my_first_repo/blob/main/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the assignment

## Website 

[Yahoo Finance](https://au.finance.yahoo.com/)

## Inputs

* Prompt for a ticker symbol. Find out what is a ticker symbol?

## Processing

* Get income from webpae.  Which page?
* Calculate Tax (using income)

## Outputs

* Save to CSV file (How, use CSV or Pandas?)
* Save to SQLIte (See week on databases)




### Exploring the Website

Stock Ticker Symbol:  A stock symbol is a unique series of letters assigned to a security for trading purposes.  From https://www.investopedia.com/terms/s/stocksymbol.asp

Ticker can be for any company on the exchange

**TODO: How to make sure ticker is valid**

Note once found page, need to click on the 'Profile' tab.

For NAB.AX the profile page is:

https://au.finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX

For BHP.AX:

https://au.finance.yahoo.com/quote/BHP.AX/profile?p=BHP.AX


For COL.AX (Coles Group)

https://au.finance.yahoo.com/quote/COL.AX/profile?p=COL.AX


Interesting, the ticker symbol appears twice in the URL.

I will need to create a template or something that I can add the ticker symbol to.

**TODO: Work out how to add the ticker symbol to the URL**


# Try to work an example by hand

Here is the table from the website for COL.AX

```
Name	Title	Pay	Exercised	Year born
Mr. Steven Cain	MD, CEO & Director	3.33M	N/A	1965
Ms. Leah Weckert	Chief Financial Officer	1.8M	N/A	N/A
Mr. Matthew Swindells	Chief Operations Officer	1.58M	N/A	N/A
Mr. Greg Davis	Chief Exec. - Commercial & Express	1.61M	N/A	N/A
Mr. John Cox	Chief Technology Officer	N/A	N/A	N/A
Mark Howell	Gen. Mang. of Investor Relations	N/A	N/A	N/A
Mr. David Brewster	Chief Legal Officer	N/A	N/A	N/A
Ms. Lisa Ronson	Chief Marketing Officer	N/A	N/A	N/A
Ms. Kris Webb	Chief People Officer	N/A	N/A	N/A
Mr. Thinus Keeve	Chief Sustainability, Property & Export Officer	N/A	N/A	N/A

```

Lecturer clarified we only need Name, Title, Pay

Let consider one row, and only name title and pay

```
Name	              Title	          Pay
Mr. Steven Cain	MD, CEO & Director	3.33M
```

Note, 1st collum is the name, the second is title and thrid is pay.

**TODO:  Work out how get each colum/cell in the table**

### Want to calculate the tax
* Need to get the pay

*Note: We did a calculate tax in a earlier lab, will revist later*

See: https://github.com/michael-borck/ISYS5002_portfolio/blob/main/tax_calculation.ipynb



```
def calculate_tax(income):
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
  elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)

  return tax_payable
```



I need the dollar amount to calculate the tax.

So Pay (or Salray) is 3.33M. Need to extract the 3.33 and multiply is by 1000,000 to give me the dollar amount.  Thne use this dollar amouunt to calculate the tax.

**TODO:  Work out where I need to save the tax payable**



In [ ]:
3.33 * 1000000

3330000.0

### Lets do another row, this time fom NAB.AX

```
Ms. Sharon J. Cook	Group Exec. of Legal & Commercial Services	912.98k
```

Notice that the pay/salary is 912.98k  So I need to extract the 912.98 and multiply by 1000.

In [ ]:
912.98 * 1000

912980.0

# Want to output...

1. First create a table

```
Name, Title, Pay, Tax
```

**TODO How do I add a column**

2. Save as CSV file

*Note:  Look at notebook to see how to save as CSV**

https://lms.curtin.edu.au/webapps/blackboard/content/listContentEditable.jsp?content_id=_9350025_1&course_id=_110320_1

3. Save as SQLite Database

*Note:  Look at notebook to see how to save as SQLite**

https://github.com/michael-borck/ISYS5002_portfolio/blob/main/sql_shell_LIVE.ipynb


